# Домашнее задание: класс линейной регрессии

## Импорт библиотек, установка констант

In [39]:
import numpy as np
import pandas as pd

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [40]:
RANDOM_STATE = 42
TEST_SIZE = 0.25

## Интерфейс Scikit-Learn

Scikit-Learn (`sklearn`)- библиотека, в которой реализованы практически все используемые сегодня алгоритмы машинного обучения.

Для реализации алгоритмов машинного обучения в `sklearn` всегда используется один интерфейс - класс с функциями `fit(X, y)` для обучения модели по обучающей выборке `X`, `y` и `predict(X)` для возвращения предсказаний на выборке `X`. При создании класса можно указывать дополнительные параметры, влияющие на работу алгоритма машинного обучения.

## Линейная регрессия

Реализуйте класс линейной регрессии c L2-регуляризацией со следующей логикой:
*   При создании класса задайте коэффициент регуляризации `reg_coef`, равный по умолчанию нулю, а также веса (=None)
*   Задача функции `fit` - по выборке `X` и `y` найти веса `w` и сохранить их внутри класса в `self.w`:  
$w = (X^TX + \lambda I)^{-1}X^Ty,$
где $\lambda$ - коэффициент регуляризации, $I$ - единичная матрица.
  
P.S. Формула верна только при наличии вектора признаков, равного 1 - поэтому для вашего удобства мы уже добавили его в класс.

*   Задача функции `predict` - по весам `self.w` и `X` вернуть предсказания  


In [41]:
class LinearRegressor:
    def __init__(self, reg_coef=0.0, weights=None) -> None:
        self.w = weights
        self.reg_coef = reg_coef

    def fit(self, X_train: np.array, y_train: np.array) -> None:
        X_train = np.hstack((np.ones((X_train.shape[0], 1)), X_train))

        I = np.eye(X_train.shape[1])

        self.w = (
            np.linalg.inv(X_train.T @ X_train + self.reg_coef * I) @ X_train.T @ y_train
        )

    def predict(self, X_test: np.array) -> np.array:
        X_test = np.hstack(
            (np.ones((X_test.shape[0], 1)), X_test)
        )  # добавление столбца единиц

        pred = X_test @ self.w  # Прогнозирование

        return pred

    def r2_score(self, X_test: np.array, y_test: np.array) -> float:
        y_pred = self.predict(X_test)
        ss_res = np.sum((y_test - y_pred) ** 2)
        ss_tot = np.sum((y_test - np.mean(y_test)) ** 2)
        return 1 - (ss_res / ss_tot)

Если бы не использовали класс, нам пришлось бы передавать веса `w` в функцию `predict()` каждый раз, когда мы захотели бы сделать предсказания, а так они хранятся внутри класса. Это особенно удобно, если таких вспомогательных переменных много.

Будем тестировать ваш класс на датасете о стоимости домов в Калифорнии.

In [42]:
X, y = fetch_california_housing(return_X_y=True, as_frame=True)

Разобъем данные на тренировочную и тестовую часть.

In [43]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE
)

Масштабируем данные.

In [44]:
sc = StandardScaler()

sc.fit(X_train)

X_train = pd.DataFrame(sc.transform(X_train), columns=X_train.columns)
X_test = pd.DataFrame(sc.transform(X_test), columns=X_test.columns)

## Задание 1

Обучите вашу линейную регрессию (без регуляризации) на тренировочных данных. Выведите на экран максимальный по модулю вес (со знаком). Ответ округлите до сотых.

In [45]:
# ваш код здесь
lr = LinearRegressor()
lr.fit(X_train, y_train)
lr.predict(X_test)
print(max(lr.predict(X_test)))
print(max(lr.w))

11.693312788839453
2.070348920542635


## Задание 2

Сделайте прогноз на тестовых данных и выведите на экран значение метрики $R^2$ на тесте.  
Ответ округлите до тысячных.

In [46]:
# ваш код здесь
lr.r2_score(X_test, y_test)

0.5910509795491352

## Задание 3

Теперь обучите линейную регрессию с коэффициентом регуляризации $\alpha = 100$ на тренировочных данных.

Чему теперь равен наибольший по модулю вес? Ответ округлите до сотых.

In [51]:
# ваш код здесь
lr_100 = LinearRegressor(reg_coef=100)
lr_100.fit(X_train, y_train)
print(max(abs(lr_100.w)))

2.0570604165596915


## Задание 4

Для модели с регуляризацией сделайте прогноз на тестовых данных и выведите на экран значение метрики $R^2$ на тесте.  
Ответ округлите до тысячных.

In [53]:
# ваш код здесь
lr_100.r2_score(X_test, y_test)

0.5924168923824344